            TODO 
-> save models to further usage
-> player

In [33]:
import pysrt
import joblib


model = joblib.load('C:\dev\whisper_models\whisper_small.pkl')

In [36]:
import os

In [39]:
import whisper
import pysrt

model = whisper.load_model("medium")

In [30]:
result = model.transcribe("C:\Movies\My Hero Academia S01 (2016) MULTi 1080p 10bits BluRay x265 AAC v2 -Punisher694\My Hero Academia S01E02 MULTi 1080p 10bits BluRay x265 AAC -Punisher694.mkv")

 Ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha! BOUM! Tout va bien se passer. Vous savez pourquoi? Ouuuuuuuuuuuh! Parce que la cavalerie est là! C'est le plus classe des superhéros! Je voudrais devenir comme lui quand j'aurai mon alter. Il s'agit d'un cas désespéré. Peut-être qu'ils ont tous raison. J'ai jamais vu un type aussi minable que toi. Tu n'as même pas d'alter. Alors tu ferais mieux d'éviter de partager mes ambitions. Tu n'es qu'un misérable boirien sans alter. Ils pensent tous la même chose de moi. Je suis désolée, mon cœur. Je suis désolée, tellement désolée. Non, maman, tu ne comprenais pas. Ce n'était pas ce que je voulais entendre. Peu importe. Peu importe ce qu'on me dira, ça ne changera rien. Est-ce qu'on peut devenir un héros même si on n'a pas d'alter ? Est-ce qu'on peut devenir comme vous ? Même quand on n'a pas de super-pouvoir ? Sans super-pouvoir ? Oh, les chi-t. C'est pas vrai. Pas maintenant. Les gens se moquent de moi. C'est peut-êt

In [4]:
def convert_time(seconds):

    hours = int(seconds // 3600)
    remaining_seconds = seconds % 3600
    
    minutes = int(remaining_seconds // 60)
    remaining_seconds %= 60
    
    seconds = int(remaining_seconds)
    miliseconds = int((remaining_seconds - seconds) * 1000)
    
    return (hours, minutes, seconds, miliseconds)


In [31]:
def result_to_srt(result, output_file):
    subs = pysrt.SubRipFile()
    
    for segment in result['segments']:
        start_time = convert_time(segment['start']) 
        end_time = convert_time(segment['end'])
        
        sub_item = pysrt.SubRipItem(start=start_time, 
                                    end=end_time, 
                                    text=segment['text'])
        subs.append(sub_item)
    subs.save(output_file, encoding='utf-8')
    print(f'Subtitle file {output_file} is ready.')   

In [32]:
result_to_srt(result, 'output2.srt')

Subtitle file output2.srt is ready.


In [37]:
def subtitles_for_folder(video_folder, sub_folder, sub_extension='.srt'):
    files = os.listdir(video_folder)
    file_count = len(files)
    done = 0
    for file in files:
        video_path = os.path.join(video_folder, file)
        result = model.transcribe(video_path)
        
        sub_file = os.path.splitext(video_path)[0] + sub_extension
        sub_path = os.path.join(sub_folder, sub_file)
        if sub_extension == '.srt':
            result_to_srt(result, sub_path)
            done += 1
            print(f"{done}/{file_count}")
        else:
            #TODO implement other subtitle formats
            print(f"Unable to create {sub_extension} files. Use '.srt' instead.")
            break
        

In [40]:
subtitles_for_folder('C:\\Users\\vlns\\Desktop\\sub_ex2', 'C:\\Users\\vlns\\Desktop\\sub_ex2')